# 과제
#### 주어진 데이터는 여러 column 정보를 이용해 자전거의 수요(count)를 예측하는 데이터입니다.  지금까지의 세션에서 알아본 내용을 바탕으로 자유롭게 모델링을 해주세요

#### 필수 포함 내용
* 오늘 알아본 회귀 모델들 전부 사용(로지스틱 회귀 제외)하고 결과 비교
* 규제가 있는 모델의 alpha 파라미터 값 변화에 따른 평가 결과 비교 (단순 값 비교, 시각화 등 자유)
* train test 분할은 train_test_split(X_features, y_target, test_size=0.2, random_state=1004)
* 평가지표는 RMSE 사용

## ++
* 본 과제는 성능보다는 오늘 알아본 모델의 사용법을 알아보는데 의의를 두었습니다. 성능이 과제의 점수를 매기진 않지만 전체에서 최고 성능을 낸 분에게 제가 커피를 사 드립니다.

### 자전거 수요 예측 데이터
    * datetime: hourly date + timestamp
    * season: 1=봄, 2=여름, 3=가을, 4=겨울
    * holiday: 1=주말을 제외한 국경일 등의 휴일, 0=휴일이 아닌 날
    * workingday: 1=주말 및 휴일이 아닌 주중, 0=주말 및 휴일
    * weather:
    * 1=맑음, 약간 구름 낀 흐림
    * 2=안개, 안개 + 흐림
    * 3=가벼운 눈, 가벼운 비 + 천둥
    * 4=심한 눈/비, 천둥/번개
    * temp: 온도(섭씨)
    * atemp: 체감온도(섭씨)
    * humidity: 상대습도
    * windspeed: 풍속
    * casual: 사전에 등록되지 않은 사용자가 대여한 횟수
    * registered: 사전에 등록된 사용자가 대여한 횟수
    * count: 대여 횟수 (target)

In [142]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [143]:
df = pd.read_csv("./data/bike.csv")

### 1) 피처 제거 및 추가

In [144]:
#필요 없는 피처 제거
df=df.drop(columns=['windspeed'],axis=1)

In [145]:
#datetime 변환
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')

In [146]:
#시간대 열 추가
df.insert(7, 'time',df['datetime'].dt.hour)

In [147]:
#datetime 열 삭제
df=df.drop(columns=['datetime'],axis=1)

In [148]:
df

,season,holiday,workingday,weather,temp,atemp,time,humidity,casual,registered,count
0,1,0,0,1,9.84,14.395,0,81,3,13,16
1,1,0,0,1,9.02,13.635,1,80,8,32,40
2,1,0,0,1,9.02,13.635,2,80,5,27,32
3,1,0,0,1,9.84,14.395,3,75,3,10,13
4,1,0,0,1,9.84,14.395,4,75,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,19.695,19,50,7,329,336
10882,4,0,1,1,14.76,17.425,20,57,10,231,241
10883,4,0,1,1,13.94,15.910,21,61,4,164,168
10884,4,0,1,1,13.94,17.425,22,61,12,117,129


### 2) 데이터 분리

In [149]:
from sklearn.model_selection import train_test_split


#타겟 데이터 선정
X_features= df.drop(columns='count')
y_target=df[['count']]

#train, test 분리
X_train , X_test, y_train , y_test= train_test_split(X_features, y_target, test_size=0.2, random_state=1004)

### 3) feature 생성

1) 시간대: 출근시간, 퇴근시간 (7~8시), (5~6시)  
2) 계절: 봄, 가을  
3) 휴일: 휴일일 때  
4) 주중: 두 가지 경우 존재 (주중=> 출퇴근) 주말: 굳  
5) 날씨: 맑음  
6) 온도: 30도 미만 영하 이상  
8) casual, registered, windspeed 삭제  

In [150]:
X_train.head()

,season,holiday,workingday,weather,temp,atemp,time,humidity,casual,registered
5849,1,0,1,1,7.38,8.335,22,47,4,74
9779,4,0,1,1,17.22,21.210,20,62,27,388
429,1,0,1,1,12.30,15.150,22,52,6,53
1554,2,0,0,1,20.50,24.240,15,77,156,141
646,1,0,1,2,5.74,6.820,4,59,0,1


In [151]:
#피처 생성
#1) 계절이 봄이거나 가을인 경우, 1로 
X_train['f1'] = [1 if i == 1 or i==2 else 0 for i in X_train['season']]
X_test['f1'] = [1 if i == 1 or i==2 else 0 for i in X_test['season']]

<ipython-input-151-4feae97a4d99>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['f1'] = [1 if i == 1 or i==2 else 0 for i in X_train['season']]
<ipython-input-151-4feae97a4d99>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['f1'] = [1 if i == 1 or i==2 else 0 for i in X_test['season']]


In [124]:
#2) 시간대가 (0~5시 새벽인 애들은 0으로 바꿔줌, 나머지는 1로) ->성능 떨어짐
#X_train['f2']= [0 if i == 1 or i==2 or i==3 or i==4 or i==5 else 0 for i in X_train['time']]
#X_test['f2']= [0 if i == 1 or i==2 or i==3 or i==4 or i==5 else 0 for i in X_test['time']]

### 4) modeling

1) 선형 회귀 2) 다항 회귀 3) 릿지 4) 라쏘 5) 엘라스틱넷

In [152]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error

In [153]:
#1) 선형 회귀
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
pred = lin_reg.predict(X_test)

MSE = mean_squared_error(y_test, pred) #실제값 , 예측값
RMSE = np.sqrt(MSE)

print('MSE :', MSE)
print('RMSE :', RMSE)

MSE : 4.974998152106734e-26
RMSE : 2.2304703880811182e-13


In [154]:
#릿지 회귀

ridge_reg = Ridge()
ridge_reg.fit(X_train, y_train)
pred = ridge_reg.predict(X_test)

MSE = mean_squared_error(y_test, pred) #실제값 , 예측값
RMSE = np.sqrt(MSE)

print('MSE :', MSE)
print('RMSE :', RMSE)


MSE : 8.86617335193465e-12
RMSE : 2.9776120217272514e-06


In [155]:
#라쏘 회귀

lasso_reg = Lasso()
lasso_reg.fit(X_train, y_train)
pred = lasso_reg.predict(X_test)

MSE = mean_squared_error(y_test, pred) #실제값 , 예측값
RMSE = np.sqrt(MSE)

print('MSE :', MSE)
print('RMSE :', RMSE)


MSE : 0.00037529225367395684
RMSE : 0.01937246121880121


In [161]:
#일라스틱넷

ElasticNet = ElasticNet()
ElasticNet.fit(X_train, y_train)

pred = ElasticNet.predict(X_test)

MSE = mean_squared_error(y_test, pred) #실제값 , 예측값
RMSE = np.sqrt(MSE)

print('MSE :', MSE)
print('RMSE :', RMSE)


MSE : 0.00038387999067357437
RMSE : 0.019592855602835805


### 5) 규제 (a값에 따른 RMSE 변화)

In [156]:
from sklearn.model_selection import cross_val_score

In [157]:
# alpha값에 따른 회귀 모델의 폴드 평균 RMSE를 출력하고 회귀 계수값들을 DataFrame으로 반환 
def get_linear_reg_eval(model_name, params=None, X_data_n=None, y_target_n=None, 
                        verbose=True, return_coeff=True):
    coeff_df = pd.DataFrame()
    if verbose : print('####### ', model_name , '#######')
    for param in params:
        if model_name =='Ridge': model = Ridge(alpha=param)
        elif model_name =='Lasso': model = Lasso(alpha=param)
        elif model_name =='ElasticNet': model = ElasticNet(alpha=param,l1_ratio=0.7)
            
        neg_mse_scores = cross_val_score(model, X_data_n, 
                                             y_target_n, scoring="neg_mean_squared_error", cv = 5)
        avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
        print('alpha {0}일 때 5 폴드 세트의 평균 RMSE: {1:.3f} '.format(param, avg_rmse))
        # cross_val_score는 evaluation metric만 반환하므로 모델을 다시 학습하여 회귀 계수 추출
       
        model.fit(X_data_n , y_target_n)

In [158]:
#ridge model a값 변화에 따른 rmse
ridge_alphas = [1000,2500,5000,10000]
coeff_ridge_df =get_linear_reg_eval('Ridge', params=ridge_alphas,
                                      X_data_n=X_features, y_target_n=y_target)

#######  Ridge #######
alpha 1000일 때 5 폴드 세트의 평균 RMSE: 0.003 
alpha 2500일 때 5 폴드 세트의 평균 RMSE: 0.008 
alpha 5000일 때 5 폴드 세트의 평균 RMSE: 0.015 
alpha 10000일 때 5 폴드 세트의 평균 RMSE: 0.030 


In [159]:
# lasso model a값 변화에 따른 rmse
lasso_alphas = [ 0.1,0.5, 1, 50, 100]
coeff_lasso_df =get_linear_reg_eval('Lasso', params=lasso_alphas, 
                                    X_data_n=X_features, y_target_n=y_target)

#######  Lasso #######
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 0.002 
alpha 0.5일 때 5 폴드 세트의 평균 RMSE: 0.012 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 0.022 
alpha 50일 때 5 폴드 세트의 평균 RMSE: 1.078 
alpha 100일 때 5 폴드 세트의 평균 RMSE: 2.157 


In [160]:
# elastic model a값 변화에 따른 rmse
elastic_alphas = [ 0.1, 0.5, 1, 3]
coeff_elastic_df =get_linear_reg_eval('ElasticNet', params=elastic_alphas, X_data_n=X_features, y_target_n=y_target)

#######  ElasticNet #######
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 0.002 
alpha 0.5일 때 5 폴드 세트의 평균 RMSE: 0.011 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 0.022 
alpha 3일 때 5 폴드 세트의 평균 RMSE: 0.065 


### RMSE

In [162]:
def get_rmse(model):
    pred = model.predict(X_test)
    mse = mean_squared_error(y_test , pred)
    rmse = np.sqrt(mse)
    print('{0} 로그 변환된 RMSE: {1}'.format(model.__class__.__name__,np.round(rmse, 3)))
    return rmse

def get_rmses(models):
    rmses = [ ]
    for model in models:
        rmse = get_rmse(model)
        rmses.append(rmse)
    return rmses

In [163]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# LinearRegression, Ridge, Lasso 학습, 예측, 평가
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)

ridge_reg = Ridge()
ridge_reg.fit(X_train, y_train)

lasso_reg = Lasso()
lasso_reg.fit(X_train, y_train)

ElasticNet = ElasticNet()
ElasticNet.fit(X_train, y_train)

models = [lr_reg, ridge_reg, lasso_reg, ElasticNet]
get_rmses(models)

LinearRegression 로그 변환된 RMSE: 0.0
Ridge 로그 변환된 RMSE: 0.0
Lasso 로그 변환된 RMSE: 0.019
ElasticNet 로그 변환된 RMSE: 0.02


[2.2304703880811182e-13,
 2.9776120217272514e-06,
 0.01937246121880121,
 0.019592855602835805]